In [2]:
# Aim of this algorithm is to get numerical value from ndvi images according to their pixel values

In [4]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import sys
import math
import pandas as pd

In [5]:
np.set_printoptions(threshold=sys.maxsize) #to see all array

In [9]:
image = cv2.imread('deneme_ndvi.tiff') # reading image array

In [10]:
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # opencv using bgr so converting bgr to rgb

In [11]:
color_identification = {"000000": 0.0,
                        'AD0028': 0.05, 
                        'C5142A': 0.1, 
                        'E02D2C': 0.15, 
                        'EF4C3A': 0.2, 
                        'FE6C4A': 0.25, 
                        'FF8D5A': 0.3, 
                        'FFAB69': 0.35, 
                        'FFC67D': 0.4, 
                        'FFE093': 0.45, 
                        'FFEFAB': 0.5, 
                        'FDFEC2': 0.55, 
                        'EAF7AC': 0.6, 
                        'D5EF94': 0.65, 
                        'B9E383': 0.7, 
                        '9BD873': 0.75, 
                        '77CA6F': 0.8, 
                        '53BD6B': 0.85, 
                        '14AA60': 0.9, 
                        '009755': 0.95, 
                        '007E47': 1.0} # custom colormap and their numerical values

In [12]:
# array to hex
image_hex_array = []
for row in image:
    each_row = []
    for column in row:
        each_row.append('#%02x%02x%02x' % tuple(column))
    image_hex_array.append(each_row)

In [13]:
#3d array to 1d arraydd
image_hex_1D = []
for row in image_hex_array:
    for column in row:
        image_hex_1D.append(column)

In [14]:
#colors goruped by colors
unique_elements, counts_elements = np.unique(image_hex_1D, return_counts=True)
image_dictionary = {}
for color in list(unique_elements):
    image_dictionary[color] = list(counts_elements)[list(unique_elements).index(color)]
print('image_dict', image_dictionary)  # image array and example array samething

image_dict {'#000000': 240, '#66bd63': 1, '#84ca66': 164, '#a4d869': 394, '#a50026': 18, '#bc1626': 11, '#bfe37a': 164, '#d62f26': 17, '#d9ef8b': 101, '#e54d34': 10, '#ecf7a5': 52, '#f46d43': 33, '#f88e52': 18, '#fcac60': 17, '#fdc675': 18, '#fee08b': 24, '#feefa5': 31, '#fefebd': 40}


In [15]:
example_array = (image_dictionary)

In [16]:
#find the closest number in a given list
def closest_hex(lst, k):
    result = {}
    color_index = tuple(int(k[i:i+2], 16) for i in (0, 2, 4))
    color_red = color_index[0]
    color_green = color_index[1]
    color_blue = color_index[2]
    for color in lst:
        ref_color = tuple(int(color[i:i+2], 16) for i in (0, 2, 4))
        ref_color_red = ref_color[0]
        ref_color_green = ref_color[1]
        ref_color_blue = ref_color[2]
        distance = math.sqrt(((ref_color_red-color_red)**2) + math.sqrt(((ref_color_green-color_green)**2)) + math.sqrt(((ref_color_blue-color_blue)**2)))
        result[distance] = color
    return result[min(list(result.keys()))]

In [17]:
# referenced colors turned into decimals
color_ind_decimal_array = [int(x, 16) for x in color_identification.keys()]
for key in example_array:
    closest_hex_val = closest_hex(list(color_identification.keys()), key[1:])
    print("input = ",key, "output =", closest_hex_val)

input =  #000000 output = 000000
input =  #66bd63 output = 77CA6F
input =  #84ca66 output = 77CA6F
input =  #a4d869 output = 9BD873
input =  #a50026 output = AD0028
input =  #bc1626 output = C5142A
input =  #bfe37a output = B9E383
input =  #d62f26 output = E02D2C
input =  #d9ef8b output = D5EF94
input =  #e54d34 output = E02D2C
input =  #ecf7a5 output = EAF7AC
input =  #f46d43 output = EF4C3A
input =  #f88e52 output = FF8D5A
input =  #fcac60 output = FFAB69
input =  #fdc675 output = FFC67D
input =  #fee08b output = FFE093
input =  #feefa5 output = FFEFAB
input =  #fefebd output = FDFEC2


In [18]:
image_ndvi_array = []
for row in image_hex_array:
    temp_row = []
    for column in row:
        closest_hex_val = closest_hex(list(color_identification.keys()), column[1:])
        temp_row.append(color_identification[closest_hex_val])
    image_ndvi_array.append(temp_row)

In [19]:
ndvi = pd.DataFrame(image_ndvi_array)

In [20]:
ndvi

,0,1,2,3,4,5,6,7,8,9,...,31,32,33,34,35,36,37,38,39,40
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.30,0.30,0.30,0.30,0.30
1,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.30,0.35,0.35,0.35,0.35,0.35,0.35,0.35,0.35,0.35
2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.45,0.50,0.55,0.55,0.55,0.50,0.50,0.50,0.55,0.55
3,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.15,0.15,...,0.55,0.60,0.60,0.60,0.60,0.55,0.55,0.55,0.60,0.65
4,0.00,0.00,0.15,0.15,0.20,0.20,0.20,0.20,0.20,0.20,...,0.65,0.65,0.70,0.70,0.70,0.65,0.65,0.65,0.70,0.70
5,0.00,0.00,0.10,0.15,0.20,0.40,0.40,0.45,0.40,0.40,...,0.75,0.70,0.75,0.75,0.75,0.70,0.70,0.70,0.75,0.75
6,0.00,0.00,0.05,0.15,0.20,0.40,0.45,0.50,0.50,0.50,...,0.75,0.75,0.75,0.75,0.75,0.70,0.70,0.70,0.75,0.75
7,0.00,0.00,0.05,0.15,0.20,0.40,0.50,0.55,0.55,0.55,...,0.75,0.75,0.75,0.75,0.75,0.70,0.70,0.70,0.75,0.75
8,0.00,0.00,0.10,0.15,0.20,0.40,0.50,0.60,0.60,0.60,...,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.00
9,0.00,0.00,0.15,0.20,0.20,0.40,0.55,0.60,0.60,0.60,...,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.00
